In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import os
import sys
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten
from tensorflow.keras.activations import relu,sigmoid
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Model

class RESNET50_MODEL:
  def __init__(self,training_data,testing_data,validation_data):
    try:

      train_formats = ImageDataGenerator(rescale=1./255,shear_range=0.2,horizontal_flip=True,zoom_range=0.2)
      test_formats= ImageDataGenerator(rescale=1./255)
      self.train_data = train_formats.flow_from_directory(training_data,
                                      target_size=(224,224),batch_size=10,class_mode='categorical')
      self.test_data =test_formats.flow_from_directory(testing_data,
                                      target_size=(224,224),batch_size=10,class_mode='categorical')
      self.valid_data = train_formats.flow_from_directory(validation_data,
                                      target_size=(224,224),batch_size=10,class_mode='categorical')

      self.labels=os.listdir(training_data)
      #print(self.train_data)

    except Exception as e:
        er_type,er_msg,line_no=sys.exc_info()
        print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')

  def model_building(self):
   try:
      self.image_size=[224,224]
      self.resnet50 = ResNet50(input_shape=self.image_size + [3], weights='imagenet', include_top=False)
      for layers in self.resnet50.layers:
          layers.trainable = False

      self.x = Flatten()(self.resnet50.output)
      self.predict = Dense(len(self.labels), activation='softmax')(self.x)
      self.model = Model(inputs=self.resnet50.inputs, outputs=self.predict)
      print(self.model.summary())
      self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
   except Exception as e:
      er_type, er_msg, line_no = sys.exc_info()
      print(f'Error in line no:{line_no.tb_lineno} and Error Message: {er_msg}')

  def training_epoch(self):
    try:
      self.model.fit(self.train_data,epochs=10,
                     validation_data=self.valid_data)
      self.model.save("/content/drive/MyDrive/DEEP LEARNING PROJECTS(VIHARATECH)/ food_classification_data1/ resnet50_outputs/resnet_model.h5")
    except Exception as e:
      er_type,er_msg,line_no=sys.exc_info()
      print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')

  def model_evaluation(self):

    try:
      output_path="/content/drive/MyDrive/DEEP LEARNING PROJECTS(VIHARATECH)/ food_classification_data1/ resnet50_outputs"
      model_path="/content/drive/MyDrive/DEEP LEARNING PROJECTS(VIHARATECH)/ food_classification_data1/ resnet50_outputs/resnet_model.h5"
      self.model_path=model_path
      self.model=load_model(self.model_path)
      self.output_path = os.path.join(output_path, "resnet_classification_report.json")
      self.class_names = np.array(list(self.test_data.class_indices.keys()))
      y_true = np.array(self.test_data.classes)
      y_pred_prob = self.model.predict(self.test_data)
      y_pred = np.argmax(y_pred_prob, axis=1)
      conf_matrix = confusion_matrix(y_true, y_pred)

      total_samples = conf_matrix.sum()
      accuracy = (np.sum(np.diag(conf_matrix))) / total_samples

      report = {}

      report["Overall_Accuracy"] = round(accuracy, 4)

      num_classes = len(self.class_names)
      for i in range(num_classes):
          TP = conf_matrix[i, i]
          FP = np.sum(conf_matrix[:, i]) - TP
          FN = np.sum(conf_matrix[i, :]) - TP
          TN = total_samples - (TP + FP + FN + TP)

          precision = TP / (TP + FP) if (TP + FP) > 0 else 0
          recall = TP / (TP + FN) if (TP + FN) > 0 else 0
          f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

          report[self.class_names[i]] = {
              "TP": int(TP),
              "TN": int(TN),
              "FP": int(FP),
              "FN": int(FN),
              "Accuracy": round((TP + TN) / (TP + TN + FP + FN), 4),
              "Precision": round(precision, 4),
              "Recall": round(recall, 4),
              "F1-Score": round(f1_score, 4)
          }

      import json
      with open(self.output_path,'w') as f:
        json.dump(report,f)

    except Exception as e:
      er_type,er_msg,line_no=sys.exc_info()
      print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')


if __name__=="__main__":
  try:
    training_data="/content/drive/MyDrive/DEEP LEARNING PROJECTS(VIHARATECH)/ food_classification_data1/food_cassify_data/training_data_150"
    testing_data="/content/drive/MyDrive/DEEP LEARNING PROJECTS(VIHARATECH)/ food_classification_data1/food_cassify_data/testing_data_20"
    validation_data="/content/drive/MyDrive/DEEP LEARNING PROJECTS(VIHARATECH)/ food_classification_data1/food_cassify_data/validation_data_30"

    obj=RESNET50_MODEL(training_data,testing_data,validation_data)
    obj.model_building()

  except Exception as e:
      er_type,er_msg,line_no=sys.exc_info()
      print(f'{er_type} <------>{er_msg}<-------->{line_no.tb_lineno}')



Found 5094 images belonging to 34 classes.
Found 680 images belonging to 34 classes.
Found 1020 images belonging to 34 classes.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 26,999,714 (103.00 MB)

 Trainable params: 3,412,002 (13.02 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

None


In [7]:
obj.training_epoch()

Epoch 1/10
<class 'NotImplementedError'> <------>numpy() is only available when eager execution is enabled.<-------->59


In [6]:
obj.model_evaluation()

68/68 ━━━━━━━━━━━━━━━━━━━━ 453s 7s/step


In [ ]:
from flask import Flask, request, jsonify
from keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image
import json

app = Flask(__name__)

# Load your models (example paths)
vgg16_model = load_model('path_to_vgg16_model.h5')
resnet50_model = load_model('path_to_resnet50_model.h5')
custom_model = load_model('path_to_custom_model.h5')

# Load the nutrients and classification report JSON files
with open('nutrients.json', 'r') as f:
    nutrients_data = json.load(f)

# Classification report JSON files
with open('vgg16_classification_report.json', 'r') as f:
    vgg16_report = json.load(f)

with open('resnet50_classification_report.json', 'r') as f:
    resnet50_report = json.load(f)

with open('custom_model_classification_report.json', 'r') as f:
    custom_model_report = json.load(f)

# List of food classes (same as in frontend)
food_classes = [
    'Pizza', 'Burger', 'Pasta', 'Ice Cream', 'Cake', 'Sandwich', 'Salad', 'Sushi', 'Soup', 'Steak',
    'Fried Chicken', 'Fish', 'Noodles', 'Donut', 'Hot Dog', 'Wrap', 'Dumplings', 'Tacos', 'Spaghetti', 'Grilled Cheese',
    'Ramen', 'Cupcakes', 'Pie', 'Fries', 'Rice', 'Chicken Wings', 'Pancakes', 'Brownie', 'Muffins', 'Curry', 'Lasagna',
    'Burrito', 'Popcorn', 'Chips', 'Spring Rolls', 'Fajitas'
]

@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['image']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    # Load image and preprocess
    img = image.load_img(file, target_size=(224, 224))  # Assuming the model expects 224x224 input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    model_name = request.args.get('model')

    # Make prediction based on selected model
    if model_name == 'vgg16':
        model = vgg16_model
        classification_report = vgg16_report
    elif model_name == 'resnet50':
        model = resnet50_model
        classification_report = resnet50_report
    elif model_name == 'custom_model':
        model = custom_model
        classification_report = custom_model_report
    else:
        return jsonify({'error': 'Invalid model name'}), 400

    # Predict using the selected model
    pred = np.argmax(model.predict(img_array), axis=-1)[0]

    # Get food name and nutrients based on prediction
    food_name = food_classes[pred]
    nutrients = nutrients_data.get(food_name, [])

    # Return prediction results
    return jsonify({
        'food_name': food_name,
        'vgg16': food_classes[np.argmax(vgg16_model.predict(img_array), axis=-1)[0]],
        'resnet50': food_classes[np.argmax(resnet50_model.predict(img_array), axis=-1)[0]],
        'custom_model': food_classes[np.argmax(custom_model.predict(img_array), axis=-1)[0]],
        'nutrients': nutrients,
        'classification_report': classification_report
    })

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Food Classification</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            display: flex;
            flex-direction: row;
            margin: 0;
            padding: 0;
        }

        .container {
            display: flex;
            flex-direction: column;
            align-items: center;
            width: 70%;
            padding: 20px;
        }

        .sidebar {
            width: 30%;
            padding: 20px;
            background-color: #f0f0f0;
            border-left: 1px solid #ccc;
            height: 100vh;
            overflow-y: auto;
        }

        .sidebar h3 {
            text-align: center;
        }

        .img-container {
            text-align: center;
            margin-bottom: 20px;
        }

        .result {
            margin-top: 20px;
            text-align: center;
        }

        .model-result {
            margin: 10px 0;
        }

        #imgUpload {
            display: block;
            margin: 0 auto;
        }

        .food-name {
            font-size: 24px;
            font-weight: bold;
        }

        .nutrient-list {
            list-style-type: none;
            padding: 0;
        }

        .nutrient-list li {
            margin: 5px 0;
        }

        .classification-report {
            margin-top: 20px;
            padding: 10px;
            background-color: #e7e7e7;
            border-radius: 5px;
            max-height: 300px;
            overflow-y: auto;
        }

        .class-names {
            padding: 10px;
            background-color: #e7e7e7;
            border-radius: 5px;
            margin-top: 20px;
            max-height: 400px;
            overflow-y: auto;
        }

        .class-names ul {
            padding: 0;
            list-style-type: none;
        }

        .class-names li {
            padding: 5px;
        }

        .choose-msg {
            text-align: center;
            color: #ff0000;
            margin-top: 15px;
        }

        select {
            padding: 10px;
            margin: 10px;
            font-size: 16px;
            width: 200px;
        }
    </style>
</head>

<body>

    <div class="container">
        <h2>Food Classification</h2>

        <!-- Dropdown to select model -->
        <div>
            <label for="modelSelect">Choose Model: </label>
            <select id="modelSelect">
                <option value="vgg16">VGG16</option>
                <option value="resnet50">ResNet50</option>
                <option value="custom_model">Custom Model</option>
            </select>
        </div>

        <div class="img-container">
            <input type="file" id="imgUpload" accept="image/*">
            <p>Upload food image for classification</p>
        </div>

        <div id="resultContainer" class="result" style="display: none;">
            <div class="food-name" id="foodName"></div>
            <div id="modelResults"></div>
            <h4>Nutrients:</h4>
            <ul id="nutrientList" class="nutrient-list"></ul>

            <!-- Classification Report Section -->
            <div id="classificationReport" class="classification-report">
                <!-- Model classification report will be displayed here -->
            </div>
        </div>
    </div>

    <div class="sidebar">
        <h3>Food Classes</h3>
        <div class="class-names">
            <ul id="foodClasses">
                <!-- Class names will be injected here -->
            </ul>
        </div>
        <p class="choose-msg">Please choose only images above</p>
    </div>

    <script>
        const foodClasses = [
            'Pizza', 'Burger', 'Pasta', 'Ice Cream', 'Cake', 'Sandwich', 'Salad', 'Sushi', 'Soup', 'Steak',
            'Fried Chicken', 'Fish', 'Noodles', 'Donut', 'Hot Dog', 'Wrap', 'Dumplings', 'Tacos', 'Spaghetti', 'Grilled Cheese',
            'Ramen', 'Cupcakes', 'Pie', 'Fries', 'Rice', 'Chicken Wings', 'Pancakes', 'Brownie', 'Muffins', 'Curry', 'Lasagna',
            'Burrito', 'Popcorn', 'Chips', 'Spring Rolls', 'Fajitas'
        ];

        // Inject food classes into the sidebar
        const foodClassesList = document.getElementById('foodClasses');
        foodClasses.forEach(item => {
            const li = document.createElement('li');
            li.textContent = item;
            foodClassesList.appendChild(li);
        });

        // Handle file upload and send to backend
        const imgUpload = document.getElementById('imgUpload');
        const modelSelect = document.getElementById('modelSelect');
        const resultContainer = document.getElementById('resultContainer');
        const foodName = document.getElementById('foodName');
        const modelResults = document.getElementById('modelResults');
        const nutrientList = document.getElementById('nutrientList');
        const classificationReport = document.getElementById('classificationReport');

        imgUpload.addEventListener('change', function(event) {
            const formData = new FormData();
            formData.append('image', event.target.files[0]);
            const selectedModel = modelSelect.value;

            // Make the API call to the backend to process the image
            fetch(`/predict?model=${selectedModel}`, {
                method: 'POST',
                body: formData
            })
            .then(response => response.json())
            .then(data => {
                // Display results
                resultContainer.style.display = 'block';
                foodName.textContent = `Predicted Food: ${data.food_name}`;

                // Display model results (classification report)
                modelResults.innerHTML = `
                    <div class="model-result">
                        <strong>VGG16 Prediction:</strong> ${data.vgg16}
                    </div>
                    <div class="model-result">
                        <strong>Custom Model Prediction:</strong> ${data.custom_model}
                    </div>
                    <div class="model-result">
                        <strong>ResNet50 Prediction:</strong> ${data.resnet50}
                    </div>
                `;

                // Display nutrients
                nutrientList.innerHTML = '';
                data.nutrients.forEach(nutrient => {
                    const li = document.createElement('li');
                    li.textContent = nutrient;
                    nutrientList.appendChild(li);
                });

                // Display classification report for the selected model
                const selectedModelReport = data.classification_report;
                classificationReport.innerHTML = `
                    <h4>Classification Report for ${selectedModel}:</h4>
                    <pre>${JSON.stringify(selectedModelReport, null, 4)}</pre>
                `;
            })
            .catch(error => {
                console.error('Error:', error);
            });
        });
    </script>

</body>

</html>
